# Лабораторна робота №5: Візуалізація даних
### *виконала студентка групи ФБ-33 Журавльова Марія*

**Мета роботи:** отримати поглиблені навички з візуалізації даних; ознайомитись з matplotlib.widgets, scipy.signal.filters, а також з Plotly, Bokeh, Altair; отримати навички зі створення інтерактивних застосунків для швидкого підбору параметрів і аналізу отриманих результатів.

**Постановка задачі:**
Створіть програму, яка дозволить користувачам малювати графік функції гармоніки (функція виду *y(t) = A ∗ sin(ω ∗ t + φ)*) з накладеним шумом та надавати можливість змінювати параметри гармоніки та шуму за допомогою інтерактивного інтерфейсу, що включає в себе слайдери, кнопки та чекбокси. Зашумлену гармоніку відфільтруйте за допомогою фільтру на вибір, порівняйте результат.


**Завдання 1:**
1. Створіть програму, яка використовує бібліотеки Matplotlib для створення графічного інтерфейсу.
2. Реалізуйте функцію harmonic_with_noise, яка приймає наступні параметри:
o amplitude - амплітуда гармоніки.
o frequency - частота гармоніки.
o phase – фазовий зсув гаромніки
o noise_mean - середнє шуму.
o noise_covariance – дисперсія шуму
o show_noise - флаг, який вказує, чи слід показувати шум на графіку.
3. У програмі має бути створено головне вікно з такими елементами інтерфейсу:
o Поле для графіку функції (plot)
o Слайдери (sliders), які відповідають за амплітуду, частоту гармоніки, а також
слайдери для параметрів шуму
o Чекбокс для перемикання відображення шуму на гармоніці
o Кнопка «Reset», яка відновлює початкові параметри
4. Програма повинна мати початкові значення кожного параметру, а також передавати параметри для відображення оновленого графіку.
5. Через чекбокс користувач може вмикати або вимикати відображення шуму на графіку. Якщо прапорець прибрано – відображати «чисту гармоніку», якщо ні – зашумлену.
6. Після оновлення параметрів програма повинна одразу оновлювати графік функції гармоніки з накладеним шумом згідно з виставленими параметрами.
o Зауваження: якщо ви змінили параметри гармоніки, але не змінювали параметри шуму, то шум має залишитись таким як і був, а не генеруватись
наново. Якщо ви змінили параметри шуму, змінюватись має лише шум – параметри гармоніки мають залишатись незмінними.
7. Після натискання кнопки «Reset», мають відновитись початкові параметри
8. Залиште коментарі та інструкції для користувача, які пояснюють, як користуватися програмою.
9. Завантажте файл зі скриптом до вашого репозиторію на GitHub
10. Надайте короткий звіт про ваш досвід та вивчені навички.
Зауваження: структура застосунку, назви функцій та змінних, навіть вибір бібліотек наведено з метою рекомендації. Ви вільні обирати це за вашим смаком, головна вимога – повне збереження функціоналу.

**Завдання 2:**
1. Отриману гармоніку з накладеним на неї шумом відфільтруйте за допомогою фільтру на ваш вибір (наприклад scipy.signal.iirfilter, повний список за посиланням: https://docs.scipy.org/doc/scipy/reference/signal.html). Відфільтрована гармоніка має бути максимально близька до «чистої»
2. Відобразіть відфільтровану «чисту» гармоніку поряд з початковою
3. Додайте відповідні інтерактивні елементи (чекбокс показу, параметри фільтру тощо) та оновіть існуючі: відфільтрована гармоніка має оновлюватись разом з початковою.

In [2]:
%matplotlib tk

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, CheckButtons
from scipy.signal import butter, filtfilt

# Початкові значення
init_amplitude = 1.0
init_frequency = 1.0
init_phase = 0.0
init_noise_mean = 0.0
init_noise_covariance = 0.1
init_cutoff = 3.0
show_noise = True

# Кешування шуму
noise_cached = None
noise_cached_mean = None
noise_cached_cov = None

# Часовий ряд
fs = 1000
t = np.linspace(0, 10, fs * 10)

# Генерація гармоніки
def generate_harmonic(amplitude, frequency, phase):
    return amplitude * np.sin(2 * np.pi * frequency * t + phase)

# Генерація шуму
def generate_noise(noise_mean, noise_covariance):
    return np.random.normal(noise_mean, np.sqrt(noise_covariance), len(t))

# Гармоніка з шумом
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise):
    global noise_cached, noise_cached_mean, noise_cached_cov
    y = generate_harmonic(amplitude, frequency, phase)

    if noise_cached is None or noise_mean != noise_cached_mean or noise_covariance != noise_cached_cov:
        noise_cached = generate_noise(noise_mean, noise_covariance)
        noise_cached_mean = noise_mean
        noise_cached_cov = noise_covariance

    return y + noise_cached if show_noise else y

# Фільтрація
def filter_signal(signal, cutoff):
    nyquist = 0.5 * fs
    norm_cutoff = cutoff / nyquist
    b, a = butter(4, norm_cutoff, btype='low')
    return filtfilt(b, a, signal)

# Графік
fig, ax = plt.subplots(figsize=(12, 6))
plt.subplots_adjust(left=0.15, bottom=0.5)
ax.set_title("Гармоніка з шумом і фільтрацією")
ax.set_xlabel("Час")
ax.set_ylabel("Амплітуда")
ax.grid(True)

# Початкові графіки
y_clean = generate_harmonic(init_amplitude, init_frequency, init_phase)
y_noisy = harmonic_with_noise(init_amplitude, init_frequency, init_phase, init_noise_mean, init_noise_covariance, show_noise)
y_filtered = filter_signal(y_noisy, init_cutoff)

line_clean, = ax.plot(t, y_clean, label='Чиста гармоніка', color='green')
line_noisy, = ax.plot(t, y_noisy, label='З шумом', color='orange')
line_filtered, = ax.plot(t, y_filtered, label='Фільтрований сигнал', color='black', linestyle='--')
ax.legend()

# Слайдери
sliders = {
    'amplitude': Slider(plt.axes([0.2, 0.4, 0.65, 0.03]), 'Amplitude', 0.1, 5.0, valinit=init_amplitude),
    'frequency': Slider(plt.axes([0.2, 0.35, 0.65, 0.03]), 'Frequency', 0.1, 5.0, valinit=init_frequency),
    'phase': Slider(plt.axes([0.2, 0.3, 0.65, 0.03]), 'Phase', 0.0, 2*np.pi, valinit=init_phase),
    'noise_mean': Slider(plt.axes([0.2, 0.25, 0.65, 0.03]), 'Noise Mean', -1.0, 1.0, valinit=init_noise_mean),
    'noise_cov': Slider(plt.axes([0.2, 0.2, 0.65, 0.03]), 'Noise Cov', 0.01, 1.0, valinit=init_noise_covariance),
    'cutoff': Slider(plt.axes([0.2, 0.15, 0.65, 0.03]), 'Cutoff Freq', 0.1, 10.0, valinit=init_cutoff)
}

# Оновлення графіка
def update(val):
    a = sliders['amplitude'].val
    f = sliders['frequency'].val
    p = sliders['phase'].val
    m = sliders['noise_mean'].val
    v = sliders['noise_cov'].val
    c = sliders['cutoff'].val

    y_clean = generate_harmonic(a, f, p)
    y_noisy = harmonic_with_noise(a, f, p, m, v, show_noise)
    y_filtered = filter_signal(y_noisy, c)

    line_clean.set_ydata(y_clean)
    line_noisy.set_ydata(y_noisy)
    line_filtered.set_ydata(y_filtered)

    fig.canvas.draw_idle()

for s in sliders.values():
    s.on_changed(update)

# Чекбокс
def toggle_noise(label):
    global show_noise
    show_noise = not show_noise
    update(None)

check = CheckButtons(plt.axes([0.05, 0.1, 0.1, 0.05]), ['Show Noise'], [show_noise])
check.on_clicked(toggle_noise)

# Reset
def reset(event):
    for s in sliders.values():
        s.reset()
    global noise_cached, noise_cached_mean, noise_cached_cov, show_noise
    noise_cached = None
    noise_cached_mean = None
    noise_cached_cov = None
    show_noise = True
    update(None)

reset_btn = Button(plt.axes([0.75, 0.1, 0.1, 0.04]), 'Reset')
reset_btn.on_clicked(reset)

plt.show()

**Завдання 3 (додаткове)**
1. Реалізуйте завдання 1 за допомогою сучасних графічних бібліотек на ваш вибір: Plotly, Bokeh, Altair тощо. Додайте декілька вікон для візуалізації замість одного, спадне меню (drop-down menu) та інші інтерактивні елементи на власний розсуд.
2. Реалізуйте ваш власний фільтр, використовуючи виключно Python (а також numpy, але виключно для операцій з масивами numpy.ndarray). Застосуйте фільтр

In [8]:
import numpy as np
from bokeh.layouts import column, row
from bokeh.models import Slider, Button, CheckboxGroup, ColumnDataSource
from bokeh.plotting import figure, curdoc

# Початкові значення
fs = 1000
t = np.linspace(0, 10, fs * 10)

init_amplitude = 1.0
init_frequency = 1.0
init_phase = 0.0
init_noise_mean = 0.0
init_noise_cov = 0.1
init_window = 30

# Кеш для шуму
noise_cache = {'data': None, 'mean': None, 'cov': None}

# Функції
def generate_harmonic(a, f, p):
    return a * np.sin(2 * np.pi * f * t + p)

def generate_noise(mean, cov):
    return np.random.normal(mean, np.sqrt(cov), len(t))

def harmonic_with_noise(a, f, p, mean, cov, show):
    global noise_cache
    y = generate_harmonic(a, f, p)
    if noise_cache['data'] is None or noise_cache['mean'] != mean or noise_cache['cov'] != cov:
        noise_cache['data'] = generate_noise(mean, cov)
        noise_cache['mean'] = mean
        noise_cache['cov'] = cov
    return y + noise_cache['data'] if show else y

def moving_average(signal, window_size):
    kernel = np.ones(window_size) / window_size
    return np.convolve(signal, kernel, mode='same')

# Джерела даних
source_clean = ColumnDataSource(data=dict(x=t, y=generate_harmonic(init_amplitude, init_frequency, init_phase)))
source_noisy = ColumnDataSource(data=dict(x=t, y=harmonic_with_noise(init_amplitude, init_frequency, init_phase, init_noise_mean, init_noise_cov, True)))
source_filtered = ColumnDataSource(data=dict(x=t, y=moving_average(source_noisy.data['y'], init_window)))

# Побудова графіків
plot_clean = figure(title="Чиста гармоніка", height=250, width=800)
r_clean = plot_clean.line('x', 'y', source=source_clean, color='green', legend_label='Clean')

plot_noisy = figure(title="Гармоніка з шумом", height=250, width=800)
r_noisy = plot_noisy.line('x', 'y', source=source_noisy, color='orange', legend_label='Noisy')

plot_filtered = figure(title="Фільтрована гармоніка", height=250, width=800)
r_filtered = plot_filtered.line('x', 'y', source=source_filtered, color='blue', line_dash='dashed', legend_label='Filtered')

# Віджети
slider_ampl = Slider(title="Амплітуда", start=0.1, end=5.0, value=init_amplitude, step=0.1)
slider_freq = Slider(title="Частота", start=0.1, end=5.0, value=init_frequency, step=0.1)
slider_phase = Slider(title="Фаза", start=0.0, end=2*np.pi, value=init_phase, step=0.1)
slider_mean = Slider(title="Середнє шуму", start=-1.0, end=1.0, value=init_noise_mean, step=0.05)
slider_cov = Slider(title="Дисперсія шуму", start=0.01, end=1.0, value=init_noise_cov, step=0.01)
slider_window = Slider(title="Розмір вікна фільтра", start=1, end=500, value=init_window, step=1)
checkbox = CheckboxGroup(labels=["Показати шум", "Показати фільтр"], active=[0, 1])
reset_btn = Button(label="Скинути", button_type="success")

# Оновлення
def update(attr, old, new):
    a = slider_ampl.value
    f = slider_freq.value
    p = slider_phase.value
    m = slider_mean.value
    v = slider_cov.value
    w = int(slider_window.value)
    show_n = 0 in checkbox.active
    show_f = 1 in checkbox.active

    y_clean = generate_harmonic(a, f, p)
    y_noisy = harmonic_with_noise(a, f, p, m, v, show_n)
    y_filtered = moving_average(y_noisy, w)

    source_clean.data = dict(x=t, y=y_clean)
    source_noisy.data = dict(x=t, y=y_noisy)
    source_filtered.data = dict(x=t, y=y_filtered)

    r_noisy.visible = show_n
    r_filtered.visible = show_f

# Скидання
def reset():
    slider_ampl.value = init_amplitude
    slider_freq.value = init_frequency
    slider_phase.value = init_phase
    slider_mean.value = init_noise_mean
    slider_cov.value = init_noise_cov
    slider_window.value = init_window
    checkbox.active = [0, 1]
    noise_cache['data'] = None
    update(None, None, None)

# Прив'язка подій
for wdg in [slider_ampl, slider_freq, slider_phase, slider_mean, slider_cov, slider_window]:
    wdg.on_change('value', update)
checkbox.on_change('active', update)
reset_btn.on_click(reset)

# Компоновка
controls = column(slider_ampl, slider_freq, slider_phase,
                  slider_mean, slider_cov, slider_window,
                  checkbox, reset_btn)
layout = column(plot_clean, plot_noisy, plot_filtered, controls)
curdoc().add_root(layout)
curdoc().title = "Інтерактивна гармоніка з фільтром"

In [10]:
!bokeh serve --show Lab5.py

^C
